In [1]:
import keras

Using TensorFlow backend.
/Users/stevejpurves/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import numpy as np

In [4]:
import pandas as pd
df = pd.read_pickle('training_set.pkl')
df.head()

,idWlb,FM,S
0,82.0,82.0,95
1,82.0,82.0,95
2,82.0,82.0,95
3,82.0,82.0,95
4,82.0,82.0,95


In [6]:

wells = [well for well in df.idWlb.drop_duplicates()]
formations = [int(f) for f in df.FM.drop_duplicates()]
print(formations)
blind_wells = [1486, 107, 1140, 6678, 874, 82, 1411, 3558, 5916, 4244]
training_wells = np.setdiff1d(wells, blind_wells)

print(len(blind_wells))
print(len(training_wells))

list_of_wells = [df.loc[df.idWlb == well,['S','FM']] for well in training_wells]
list_of_wells[0].head()

to_formation_id = np.vectorize(lambda x: formations.index(x))

input_sequences = [np.expand_dims(df['S'].values.astype(dtype=np.int), axis=1) for df in list_of_wells]
output_sequences = [np.expand_dims(to_formation_id(df['FM'].values.astype(dtype=np.int)), axis=1) for df in list_of_wells]

[82, 63, 159, 118, 174, 46, 150, 83, 47]


NameError: name 'np' is not defined

In [54]:
batch_size = 1
time_step = 5

from keras.preprocessing.sequence import TimeseriesGenerator

training_set = []
for symbols, targets in zip(input_sequences, output_sequences):
    training_set.append(TimeseriesGenerator(symbols, targets,
                               length=time_step,
                               sampling_rate=1,
                               batch_size=batch_size))
print(len(training_set))

43


In [55]:
input = Input(batch_shape=(batch_size, time_step, 1))
lstm = LSTM(latent_dim,
            batch_input_shape=(batch_size, time_step, 1))(input)
output = Dense(1)(lstm)

In [56]:
model = Model(input, output)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (1, 5, 1)                 0         
_________________________________________________________________
lstm_12 (LSTM)               (1, 256)                  264192    
_________________________________________________________________
dense_2 (Dense)              (1, 1)                    257       
Total params: 264,449
Trainable params: 264,449
Non-trainable params: 0
_________________________________________________________________


In [58]:
checkpoint = ModelCheckpoint('./1xLSTM_intervals.acc.hdf5', monitor='acc', verbose=1, save_best_only=True, mode='max')

num_iter = 100
history = []

for i in range(0, num_iter):
    print("iter", i)
    for w, well in enumerate(training_set):
        print("well", w)
        history.append(model.fit_generator(well,
                      epochs=1,
                      callbacks=[checkpoint]))

iter 0
well 0
Epoch 1/1
1634/1634 [==============================] - 9s 6ms/step - loss: 0.2822
well 1
Epoch 1/1
   19/10756 [..............................] - ETA: 1:06 - loss: 14.6978

/Users/stevejpurves/anaconda3/lib/python3.5/site-packages/keras/callbacks.py:435: RuntimeWarning: Can save best model only with acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


10756/10756 [==============================] - 65s 6ms/step - loss: 4.5675
well 2
Epoch 1/1
14467/14467 [==============================] - 81s 6ms/step - loss: 5.6554
well 3
Epoch 1/1
8735/8735 [==============================] - 52s 6ms/step - loss: 0.0147
well 4
Epoch 1/1
6228/6228 [==============================] - 34s 5ms/step - loss: 3.2114
well 5
Epoch 1/1
14545/14545 [==============================] - 82s 6ms/step - loss: 0.1876
well 6
Epoch 1/1
7218/7218 [==============================] - 41s 6ms/step - loss: 0.8791
well 7
Epoch 1/1
13183/13183 [==============================] - 75s 6ms/step - loss: 1.2284
well 8
Epoch 1/1
16069/16069 [==============================] - 92s 6ms/step - loss: 2.1039
well 9
Epoch 1/1
7619/7619 [==============================] - 43s 6ms/step - loss: 0.8165
well 10
Epoch 1/1
9574/9574 [==============================] - 55s 6ms/step - loss: 1.7652
well 11
Epoch 1/1
10901/10901 [==============================] - 64s 6ms/step - loss: 1.3904
well 12
Epoch

KeyboardInterrupt: 